# Object Tracking & Intent Analysis (v7 - Interaction Distance, Full Annotations)

In [17]:
# Install Ultralytics (YOLOv8) and BoxMOT
!pip install ultralytics --quiet
!pip install boxmot --quiet

# Import necessary libraries
import os
import cv2
import time
import yaml
import torch
import json
import numpy as np
from pathlib import Path
from collections import defaultdict, deque
from ultralytics import YOLO
import datetime
# from google.colab.patches import cv2_imshow # Usually not needed if running locally or if cv2.imshow works

# Define DummyTracker globally for fallback

class DummyTracker:
    def __init__(self, *args, **kwargs):
        print("Initialized DummyTracker for BoxMOT fallback.")
        self.frame_id = 0
    def update(self, dets, img): # img argument is often expected by trackers
        self.frame_id += 1
        if dets is None or len(dets) == 0:
            return np.empty((0, 7)) # BoxMOT ByteTrack returns 7 columns: x1,y1,x2,y2,id,cls,conf
        
        if hasattr(dets, 'cpu') and not isinstance(dets, np.ndarray):
            dets_np = dets.cpu().numpy()
        else:
            dets_np = dets
            
        fake_tracks = []
        for i, det_row in enumerate(dets_np):
            x1, y1, x2, y2 = det_row[:4]
            conf = det_row[4] if len(det_row) > 4 else 0.5 
            cls = det_row[5] if len(det_row) > 5 else (i % 5) # Placeholder class
            fake_tracks.append([x1, y1, x2, y2, self.frame_id * 1000 + i, cls, conf])
        return np.array(fake_tracks)


# Attempt to import BoxMOT and its utilities
try:
    from boxmot import create_tracker
    from boxmot.utils import TRACKER_CONFIGS
    print("BoxMOT and create_tracker imported successfully.")
    if TRACKER_CONFIGS is None:
        print("WARNING: boxmot.utils.TRACKER_CONFIGS is None. main() will attempt to find config path manually.")
except ImportError as e:
    print(f"ERROR: Failed to import BoxMOT: {e}")
    print("Please ensure BoxMOT is installed: pip install boxmot")
    print("WARNING: BoxMOT not available. Real tracking will not work. Falling back to DummyTracker.")
    create_tracker = lambda *args, **kwargs: DummyTracker(*args, **kwargs)
    TRACKER_CONFIGS = Path("dummy_boxmot_configs") # Relative path for dummy config
    if not TRACKER_CONFIGS.exists():
        TRACKER_CONFIGS.mkdir(parents=True, exist_ok=True)


BoxMOT and create_tracker imported successfully.


## Configuration Parameters & Global Variables

In [18]:
# --- Configuration & Global Variables ---
# ROI_MODE: 0 = Manual ROI, 1 = Dynamic ROI from frame margin
ROI_MODE = 0 
ROI_MARGIN_PIXELS = 10 # Margin in pixels for dynamic ROI (if ROI_MODE = 1)
MANUAL_ROI = (0, 250, 650, 720) # Manual ROI: (x1, y1, x2, y2) or None (if ROI_MODE = 0), too big ROI will cause ROI exit harder 

# Behavior analysis parameters
LOITERING_THRESHOLD_SEC = 5 # Seconds of inactivity to trigger loitering
INTERACTION_PROXIMITY_THRESHOLD = 210 # Distance in pixels for interaction proximity
TRACK_HISTORY_LENGTH = 60 # Frames of history for behavior analysis
EVENT_LOG_FILE = "event_log_boxmot_v8.json" # Relative path

# Event Media Saving Parameters
ENABLE_EVENT_CLIPS = False
ENABLE_EVENT_SNAPSHOTS = True
# EVENT_CLIP_OUTPUT_DIR = "event_clips" # Relative path, will be created if not exists
# EVENT_SNAPSHOT_OUTPUT_DIR = "event_snapshots" # Relative path, will be created if not exists
LOITERING_EVENT_CLIP_PRE_BUFFER_SEC = 2
LOITERING_EVENT_CLIP_POST_BUFFER_SEC = 1 # Per user request, loitering clip ends at event time
INSTANT_EVENT_CLIP_TOTAL_DURATION_SEC = 10 # Centered around event time (5s before, 5s after)
FRAME_BUFFER_DURATION_SEC = 10 # Max duration of ANNOTATED frames to keep in memory

# Global data structures
ROI = None
track_history = defaultdict(lambda: deque(maxlen=TRACK_HISTORY_LENGTH))
object_loitering_start_time = defaultdict(lambda: None)
# 新增：追蹤物件是否在ROI內的狀態字典
object_in_roi_status = defaultdict(lambda: False)
# 新增：追蹤物件之間的互動狀態字典
object_interaction_status = defaultdict(lambda: {})
event_log = [] # Will be populated by log_event, and saved at the end

# Frame buffer for video clip saving (stores (ANNOTATED_frame_copy, timestamp))
frame_buffer = deque()
active_clip_capture_tasks = []

# Statistics counters
total_frames_read_count = 0
total_frames_processed_count = 0
cumulative_detected_class_counts = defaultdict(int)

# ------------------------------------------------------------------
# 全域狀態（請放到檔案最上方一次宣告）
pair_state = {}                    # {(id_small,id_big): 'near'/'far'}
INTER_THRESH_RATIO = 0.15          # 中心距離 < 影像對角線 * 15% 視為 near

# 允許計算互動的「類別對」；None = 所有組合都計算
ALLOWED_INTERACTIONS = {
    ("other_person", "package"),
    ("other_person", "bag"),
    ("delivery_worker", "package"),
    ("delivery_worker", "bag"),
    ("food_delivery", "package"),
    ("food_delivery", "bag"),
}
# ------------------------------------------------------------------
# －－ constants －－
ARRIVAL_AWAY_DIST_RATIO = 0.10          # 兩物件同時運動時的中心距離threshold，影像對角線 * 10 %
ROI_EVENT_WINDOW_FRAMES  = 30            # 幾幀內算「同時」
ARRIVAL_DIST_PX   = 150   # <= 可自行調整
AWAY_DIST_PX      = 150
FRAME_WINDOW_SECOND      = 5    # 多少秒以內視為「同時」
# -------------------

# 存最近幾幀發生的 ROI 進出事件
recent_roi_enters = deque(maxlen=30)    # 每項: {'id':..,'cls':..,'cent':(x,y),'frame_idx':..}
recent_roi_exits  = deque(maxlen=30)
approach_depature_thresh_px = 210 # 判斷的距離閾值
arrival_away_thresh_px = ARRIVAL_DIST_PX # 事件判斷的距離閾值

# ── 觸發後不再重複的配對集合
something_arrival_pairs: set[tuple[int, int]] = set()

# ── 畫面文字維持時間：pair_key ➜ expire_ts
arrival_overlay: dict[tuple[int, int], datetime.datetime] = {}

# ── 顯示多久 (秒) FRAME_WINDOW_SECOND 請與既有程式保持一致
ARRIVAL_DISPLAY_SEC = 3


# 取代原先 pair_key→{'pt', 'expire'} 的做法
away_overlay: dict[int, datetime.datetime] = {}   # keep until expire
AWAY_DISPLAY_SEC = 3                              # 顯示秒數


In [19]:
# ===== Path Helper (放在 Notebook 最前面即可) =====================
# from pathlib import Path
from datetime import datetime

# ▶ 修改這兩行就能切換資料來源與輸出根目錄
INPUT_SOURCE = "golden_sample.mp4"      # 或 RTSP/HTTP URL
OUTPUT_ROOT  = Path("output")               # 建議集中管理

def init_paths(input_path: str | Path, add_timestamp: bool = True):
    """依輸入檔名自動建立版本化輸出目錄與全域變數。"""
    global RUN_NAME, RUN_DIR, EVENT_LOG_FILE
    global EVENT_CLIP_OUTPUT_DIR, EVENT_SNAPSHOT_OUTPUT_DIR

    input_path = Path(str(input_path))
    RUN_NAME   = input_path.stem
    ts_layer   = datetime.now().strftime("%Y%m%d_%H%M%S") if add_timestamp else ""
    RUN_DIR    = OUTPUT_ROOT / RUN_NAME / ts_layer

    EVENT_CLIP_OUTPUT_DIR     = RUN_DIR / "clips"
    EVENT_SNAPSHOT_OUTPUT_DIR = RUN_DIR / "snapshots"
    EVENT_LOG_FILE            = RUN_DIR / f"{RUN_NAME}_events.json"

    # 建立必要目錄
    EVENT_CLIP_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    EVENT_SNAPSHOT_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    print("▍Path initialised")
    print(" RUN_DIR                  :", RUN_DIR)
    print(" EVENT_CLIP_OUTPUT_DIR    :", EVENT_CLIP_OUTPUT_DIR)
    print(" EVENT_SNAPSHOT_OUTPUT_DIR:", EVENT_SNAPSHOT_OUTPUT_DIR)
    print(" EVENT_LOG_FILE           :", EVENT_LOG_FILE)

# ★ 呼叫一次，之後整支 Notebook 都能用全域變數
init_paths(INPUT_SOURCE)


▍Path initialised
 RUN_DIR                  : output\golden_sample\20250601_134922
 EVENT_CLIP_OUTPUT_DIR    : output\golden_sample\20250601_134922\clips
 EVENT_SNAPSHOT_OUTPUT_DIR: output\golden_sample\20250601_134922\snapshots
 EVENT_LOG_FILE           : output\golden_sample\20250601_134922\golden_sample_events.json


In [20]:
LINE_CHANNEL_ACCESS_TOKEN='UZE4h+sLvPk/7ueINMMK+I/AoqyUIj9apJhk+clU0RHL2MzUu2YB9Whqt/zIvREQS8dJJxM0BEk6/T/zC6DQwfBT5xz+I2v6pmMm2996+d3r9uOj9T+4rw5RluMoSB9NqantoYxGXjfWY+oRexwTSQdB04t89/1O/w1cDnyilFU='
LINE_TARGET_ID='U0b438da7f84a28c344738f3e3a1c3238'
GCS_BUCKET_NAME='cv_event_image'

In [21]:
import cv2, os, tempfile, requests, pyimgur
from dotenv import load_dotenv
# from linebot import LineBotApi
# import linebot.v3.messaging
from linebot.v3.messaging import MessagingApi, Configuration, ApiClient
from linebot.v3.messaging.models import ImageMessage, TextMessage, PushMessageRequest

# access google storage 
from google.cloud import storage
from google.cloud.exceptions import NotFound
from google.oauth2 import service_account

storage_cred = service_account.Credentials.from_service_account_file(
    'storage-compute-key.json')
storage_client = storage.Client(credentials=storage_cred)


load_dotenv()
LINE_TOKEN   = LINE_CHANNEL_ACCESS_TOKEN
LINE_TARGET  = LINE_TARGET_ID

linebot_configuration = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)

# line_api = MessagingApi(LINE_TOKEN) if LINE_TOKEN else None

def upload_image_to_gcs(local_image_path, gcs_image_name):
    # storage_client = storage.Client()
    bucket = storage_client.bucket(GCS_BUCKET_NAME)
    blob = bucket.blob(gcs_image_name)

    # blob.upload_from_filename(local_image_path)
    # blob.make_public()  # 設為公開
    # 嘗試刪除舊文件（如果存在）
    try:
        blob.delete()
    except NotFound:
        pass  # 如果文件不存在，忽略錯誤
    blob.upload_from_filename(local_image_path)
    # 生成公開訪問 URL（設置為24小時後過期）
    url = blob.generate_signed_url(
        version="v4",
        expiration=datetime.timedelta(hours=24),
        method="GET"
    )
    return url

def push_image_to_line(user_id, image_url,text=None):
    with ApiClient(linebot_configuration) as api_client:
        line_bot = MessagingApi(api_client)
        
        messages = []
        if text:
            messages.append(TextMessage(text=text))
        messages.append(ImageMessage(
            original_content_url=image_url,
            preview_image_url=image_url
        ))

        push_message = PushMessageRequest(
            to=user_id,
            messages=messages
        )
        line_bot.push_message(push_message)

def push_line_image(msg: str,event_name: str, frame_bgr):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    gcs_image_name = f"event_images/{event_name}_{timestamp}.jpg"

    # 保存暫存圖
    fd, tmpjpg = tempfile.mkstemp(suffix=".jpg")
    os.close(fd)
    cv2.imwrite(tmpjpg, frame_bgr)
    
    url=upload_image_to_gcs(tmpjpg, gcs_image_name)
    push_image_to_line(LINE_TARGET, url, text=msg)
    print(f"[LINE-Bot] 圖片已推送到 LINE: {url}")
    os.remove(tmpjpg)


In [22]:
# -----------------------------------------------
# utils/annot_line.py  （或任一共用模組）
# -----------------------------------------------
import cv2
# from line_push import push_line_image   # ← 確保已採用新 LINE Messaging API 方案

EVENT_COLORS = {
    "arrival": (255,   0, 255),   # 粉紫
    "away"   : (255, 255,   0)    # 淡黃
}

def check_and_emit(timestamp,
                   event_name,
                   obj1_id,
                   obj2_id,
                   frame_w,
                   frame_h,
                   annotated_frame,
                   line_prefix: str = "⚠️"):
    """
    只做兩件事：
    1) 在 annotated_frame 畫 event_name 文字
    2) 將影像與訊息傳到 LINE

    Parameters
    ----------
    event_name : "something_arrival" | "something_away"
    obj1_cent, obj2_cent : (x, y)  兩物件 centroid
    annotated_frame : numpy.ndarray (BGR)
    line_prefix : str  (預設 "⚠️")
    """
    if event_name not in EVENT_COLORS:
        raise ValueError(f"Unknown event_name: {event_name}")

    # 取得物件 class name（如有全域 class_names_dict 可用，否則略過）
    obj1_cls = None
    obj2_cls = None
    try:
        # 嘗試從 recent_roi_enters 找 class name
        global recent_roi_enters
        for obj in recent_roi_enters:
            if obj.get('id') == obj1_id:
                obj1_cls = obj.get('cls')
            if obj.get('id') == obj2_id:
                obj2_cls = obj.get('cls')
    except Exception:
        pass

    # 組合訊息
    msg = f"{line_prefix} {event_name}\n"
    msg += f"time: {timestamp}\n"
    msg += f"obj1: {obj1_id}\n"
    msg += f"obj2: {obj2_id}"
    # msg += f"obj1: {obj1_id} ({obj1_cls})\n"
    # msg += f"obj2: {obj2_id} ({obj2_cls})"

    # --- LINE 推送 ---
    push_line_image(msg, event_name, annotated_frame)


## Helper Functions (Analysis, Drawing, Logging, Media Saving)

In [23]:
def get_centroid(bbox):
    x1, y1, x2, y2 = bbox[:4]
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

def is_within_roi(centroid, current_roi):
    if current_roi is None: return False
    cx, cy = centroid; rx1, ry1, rx2, ry2 = current_roi
    return rx1 <= cx <= rx2 and ry1 <= cy <= ry2

def calculate_distance(p1, p2): return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def save_event_snapshot(annotated_frame, event_type, track_id, event_timestamp):
    if annotated_frame is None:
        print("No frame provided for snapshot.")
        return
    try:
        snap_dir = Path(EVENT_SNAPSHOT_OUTPUT_DIR)
        snap_dir.mkdir(parents=True, exist_ok=True)
        ts_str = event_timestamp.strftime("%Y%m%d_%H%M%S_%f")[:-3]
        filename_parts = [event_type]
        if track_id is not None: filename_parts.append(f"id{track_id}")
        filename_parts.append(ts_str)
        filename = "_".join(map(str, filename_parts)) + ".jpg"
        filepath = snap_dir / filename
        cv2.imwrite(str(filepath), annotated_frame)
        print(f"Event snapshot saved: {filepath}")
    except Exception as e:
        print(f"ERROR saving event snapshot: {e}")

def save_video_clip(frames_to_save, output_path_str, fps, frame_width, frame_height):
    if not frames_to_save:
        print(f"No frames to save for {output_path_str}.")
        return
    output_path = Path(output_path_str)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))
    for frame in frames_to_save:
        writer.write(frame)
    writer.release()
    print(f"Event clip saved: {output_path}")

def log_event(event_data, annotated_frame_for_media):
    global event_log, active_clip_capture_tasks, frame_buffer
    global recent_roi_enters, recent_roi_exits,total_frames_processed_count
    
    # Basic log entry structure from event_data
    log_entry = {
        "timestamp": event_data['event_timestamp'].strftime("%Y-%m-%d %H:%M:%S.%f")[:-3],
        "event_type": event_data['event_type'],
        "track_id": int(event_data['track_id']) if event_data.get('track_id') is not None else None,
        "class_name": event_data.get('class_name'),
        "details": event_data.get('details') or {}
    }
    if log_entry["class_name"] == "unknown": 
        print(f"WARNING: Detected class names 'Unknown'.")
        print(f"log_entry = {log_entry}")
    event_log.append(log_entry)
       
    if event_data['event_type'] in ('arrival', 'away'):
        details = event_data.get('details', {})
        # For something_arrival, use human_id/thing_id; for something_away, use id1/id2
        id1 = details.get('id1') or details.get('human_id')
        id2 = details.get('id2') or details.get('thing_id')
        if id1 is not None and id2 is not None:
            # 在 snapshot 前補畫 AWAY/ARRIVAL
            # 預設為畫面中心點
            frame_h, frame_w = annotated_frame_for_media.shape[:2]
            center_pt = (frame_w // 2, frame_h // 2)
            cent1 = center_pt
            cent2 = center_pt
            for obj in recent_roi_enters:
                if obj.get('id') == id1:
                    cent1 = obj.get('cent')
                if obj.get('id') == id2:
                    cent2 = obj.get('cent')
            if cent1 and cent2:
                mid_pt = (int((cent1[0] + cent2[0]) / 2), int((cent1[1] + cent2[1]) / 2))
                label = "ARRIVAL" if event_data['event_type'] == "arrival" else "AWAY"
                color = (0, 0, 255) if label == "ARRIVAL" else (0, 0, 255)
                cv2.putText(annotated_frame_for_media, label, (mid_pt[0] - 35, mid_pt[1] - 28),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            # Emit the event to LINE or other media
            check_and_emit(
                log_entry["timestamp"],
                event_data['event_type'],
                id1,
                id2,
                event_data['frame_w'],
                event_data['frame_h'],
                annotated_frame_for_media,
            )
    # Save snapshot if enabled, using the provided fully annotated frame # type: ignore
    if ENABLE_EVENT_SNAPSHOTS and annotated_frame_for_media is not None:
        save_event_snapshot(annotated_frame_for_media, event_data['event_type'], event_data.get('track_id'), event_data['event_timestamp'])

    # Create video clip task if enabled
    if ENABLE_EVENT_CLIPS:
        desired_clip_start_ts, desired_clip_end_ts = None, None
        event_timestamp = event_data['event_timestamp']
        event_type = event_data['event_type']
        track_id = event_data.get('track_id')
        details = event_data.get('details', {})
        current_fps = event_data['current_fps']
        frame_w = event_data['frame_w']
        frame_h = event_data['frame_h']

        ts_str = event_timestamp.strftime("%Y%m%d_%H%M%S_%f")[:-3]
        clip_name_parts = [event_type]

        if event_type == "loitering":
            loiter_start_time = object_loitering_start_time.get(track_id) # Assumes object_loitering_start_time is globally updated
            if loiter_start_time:
                desired_clip_start_ts = loiter_start_time - datetime.timedelta(seconds=LOITERING_EVENT_CLIP_PRE_BUFFER_SEC)
                desired_clip_end_ts = event_timestamp + datetime.timedelta(seconds=LOITERING_EVENT_CLIP_POST_BUFFER_SEC) # POST_BUFFER_SEC is 0
                if track_id is not None: clip_name_parts.append(f"id{track_id}")
        elif event_type in ["roi_enter", "roi_exit", "interaction"]:
            half_duration = datetime.timedelta(seconds=INSTANT_EVENT_CLIP_TOTAL_DURATION_SEC / 2)
            desired_clip_start_ts = event_timestamp - half_duration
            desired_clip_end_ts = event_timestamp + half_duration
            if track_id is not None: clip_name_parts.append(f"id{track_id}")
            if event_type == "interaction":
                p_id = details.get("person_id"); pkg_id = details.get("package_id")
                if p_id is not None: clip_name_parts.append(f"p{p_id}")
                if pkg_id is not None: clip_name_parts.append(f"pkg{pkg_id}")
        
        if desired_clip_start_ts and desired_clip_end_ts:
            clip_name_parts.append(ts_str)
            filename = "_".join(map(str, clip_name_parts)) + ".mp4"
            output_filepath = Path(EVENT_CLIP_OUTPUT_DIR) / filename

            task = {
                'log_entry_ts': log_entry['timestamp'], # Use the string timestamp from log_entry
                'desired_clip_start_ts': desired_clip_start_ts,
                'desired_clip_end_ts': desired_clip_end_ts,
                'collected_frames': [],
                'output_filename': str(output_filepath),
                'fps': current_fps, 'width': frame_w, 'height': frame_h,
                'header_printed': False
            }
            # Pre-fill with ANNOTATED frames already in buffer
            for f_in_buf, ts_in_buf in list(frame_buffer):
                if ts_in_buf >= desired_clip_start_ts and ts_in_buf <= event_timestamp: # Collect up to current event time
                    task['collected_frames'].append((f_in_buf, ts_in_buf))
            active_clip_capture_tasks.append(task)


def analyze_behavior(track_id,
                     history,
                     current_bbox,
                     class_id,
                     class_name,
                     current_ts,
                     fps_val,
                     current_roi,
                     f_w,
                     f_h,
                     annotated_frame=None):
    """
    依單一物件行為判斷 ROI enter / exit / loitering，
    以及離開 ROI 時觸發 something_away。  
    觸發 away 後，AWAY 文字會隨 paired 物件位置持續顯示 AWAY_DISPLAY_SEC 秒。
    -------------------------------------------------------------------------
    `annotated_frame` 若為 None，函式仍能運作但不會畫標註（向下相容）。
    """
    # ---------- 全域 ----------
    global object_loitering_start_time, object_in_roi_status, recent_roi_enters
    global total_frames_read_count, total_frames_processed_count, cumulative_detected_class_counts
    global away_overlay                                                   # ★ 新增

    events_to_log = []
    if not history or current_roi is None:
        return events_to_log

    # ──────────────────────────────────────────────────────────────
    # 0️⃣ 先修正 boxMOT 早期誤判的 class —— 只要 track_id 已存在，就用最新 class_name 覆蓋
    # ──────────────────────────────────────────────────────────────
    existing_item = next((it for it in recent_roi_enters if it['id'] == track_id), None)
    if existing_item:
        if existing_item['cls'] != class_name:
            existing_item['cls'] = class_name      # ← 更新
    already_in_recent = existing_item is not None
    # ──────────────────────────────────────────────────────────────
    # ── step-0：若此物件目前在 away_overlay，先畫 AWAY 且檢查是否過期 ──
    if annotated_frame is not None and track_id in away_overlay:
        expire_ts = away_overlay[track_id]
        if current_ts > expire_ts:                         # 已過期 → 移除
            del away_overlay[track_id]
        else:
            cx, cy = get_centroid(current_bbox)
            cv2.putText(annotated_frame, "AWAY",
                        (cx - 25, cy - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # ── 主要行為邏輯 ────────────────────────────────────────────────
    centroid = get_centroid(current_bbox)
    event_data_template = {
        'track_id': track_id,
        'class_name': class_name,
        'current_fps': fps_val,
        'frame_w': f_w,
        'frame_h': f_h
    }

    # 目前是否在 ROI
    is_in_roi_now = is_within_roi(centroid, current_roi)
    was_in_roi = object_in_roi_status.get(track_id, False)

    # ---------- ROI ENTER ----------
    # already_in_recent = any(it['id'] == track_id for it in recent_roi_enters)
    if is_in_roi_now and not was_in_roi:
        object_loitering_start_time[track_id] = current_ts
        object_in_roi_status[track_id] = True

        if not already_in_recent:
            events_to_log.append({
                **event_data_template,
                'event_timestamp': current_ts,
                'event_type': "roi_enter",
                'details': {"roi": current_roi, 'centroid': centroid}
            })
            # 新增至 recent_roi_enters
            recent_roi_enters.append({
                'enter_ts': current_ts,
                'id': track_id,
                'cls': class_name,
                'cent': centroid,
                'frame_idx': total_frames_processed_count,
                'is_loitering': False,
                'paired': None
            })

    # ---------- LOITERING ----------
    elif is_in_roi_now and was_in_roi:
        start_ts = object_loitering_start_time.get(track_id)
        if start_ts:
            dwell = (current_ts - start_ts).total_seconds()
            if dwell >= LOITERING_THRESHOLD_SEC:
                for it in recent_roi_enters:
                    if it['id'] == track_id and not it['is_loitering']:
                        it['is_loitering'] = True
                        events_to_log.append({
                            **event_data_template,
                            'event_timestamp': current_ts,
                            'event_type': "loitering",
                            'details': {
                                "duration_sec": round(dwell, 1),
                                "roi": current_roi,
                                'centroid': centroid
                            }
                        })
                        break

    # ---------- ROI EXIT ----------
    elif not is_in_roi_now and was_in_roi:
        events_to_log.append({
            **event_data_template,
            'event_timestamp': current_ts,
            'event_type': "roi_exit",
            'details': {"roi": current_roi, 'centroid': centroid}
        })

        # 從 recent_roi_enters 拿出此物件
        exiting_item = None
        for it in list(recent_roi_enters):
            if it['id'] == track_id:
                exiting_item = it
                recent_roi_enters.remove(it)
                break

        # 若 leaving 物件是「人」且曾配對 → 觸發 something_away
        if exiting_item and exiting_item['cls'] in ('other_person', 'delivery_worker', 'food_delivery'):
            paired_id = exiting_item.get('paired')
            if paired_id is not None:
                # 1) log 事件
                events_to_log.append({
                    **event_data_template,
                    'track_id': None,                  # away 與人分開
                    'event_timestamp': current_ts,
                    'event_type': "away",
                    'details': {
                        'id1': track_id,
                        'id2': paired_id,
                        'distance_px': None
                    }
                })
                # 2) 清掉包裹條目的 paired
                for it in recent_roi_enters:
                    if it['id'] == paired_id:
                        it['paired'] = None
                        break
                # 3) ★ 將包裹加進 away_overlay，之後自動畫 AWAY ★
                away_overlay[track_id] = current_ts + datetime.timedelta(seconds=AWAY_DISPLAY_SEC)

        # 清理狀態
        object_in_roi_status[track_id] = False
        object_loitering_start_time.pop(track_id, None)

    # ---------- ROI 外且之前也在 ROI 外 ----------
    # else: 不需處理

    return events_to_log


def analyze_interactions_for_frame(trk_objs,
                                   current_ts,
                                   annotated_frame,
                                   cls_names,
                                   fps_val,
                                   f_w,
                                   f_h,
                                   allowed_pairs=ALLOWED_INTERACTIONS):
    """
    判斷同一幀內所有物件的接近 / 離開事件；並在「approach」當下，
    若符合時窗條件，額外觸發 something_arrival。
    """
    global pair_state, approach_depature_thresh_px, recent_roi_enters
    global something_arrival_pairs, arrival_overlay          # ★ 新增
    events_to_log = []
    thresh_px = approach_depature_thresh_px  # 例如 210 px

    # ── step-0：先把仍在有效期內的 ARRIVAL 標註畫上去 ────────────────
    expired = []
    for pair_key, expire_ts in arrival_overlay.items():
        if current_ts > expire_ts:          # 到期就等會一起移除
            expired.append(pair_key)
            continue
        id_a, id_b = pair_key
        # 兩個物件還在畫面才畫標註
        objs_cent = {}
        for obj in trk_objs:
            _, _, _, _, tid, cls_id = obj[:6]
            if tid in pair_key:
                cx = int((obj[0] + obj[2]) / 2)
                cy = int((obj[1] + obj[3]) / 2)
                objs_cent[int(tid)] = (cx, cy)
        if len(objs_cent) == 2:
            mid_pt = (int((objs_cent[id_a][0] + objs_cent[id_b][0]) / 2),
                      int((objs_cent[id_a][1] + objs_cent[id_b][1]) / 2))
            cv2.putText(annotated_frame, "ARRIVAL",
                        (mid_pt[0] - 35, mid_pt[1] - 28),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    # 清理由於到期或配對消失的項目
    for pk in expired:
        arrival_overlay.pop(pk, None)

    # ── step-1：把所有物件整理成 dict 方便後續 ─────────────────────
    objects = {}
    for obj in trk_objs:
        x1, y1, x2, y2, tid, cls_id = obj[:6]
        tid = int(tid)
        cls_name = cls_names.get(int(cls_id), "unknown").lower()
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)
        objects[tid] = {"cent": (cx, cy), "cls": cls_name}

    # ── step-2：兩兩配對檢查 approach / depart ─────────────────────
    ids = list(objects.keys())
    for i in range(len(ids)):
        for j in range(i + 1, len(ids)):
            id1, id2 = ids[i], ids[j]
            o1, o2 = objects[id1], objects[id2]

            # 只保留有意義的配對
            if allowed_pairs is not None:
                pair_cls = (o1["cls"], o2["cls"])
                if pair_cls not in allowed_pairs and pair_cls[::-1] not in allowed_pairs:
                    continue

            # 距離 / 狀態
            dist = calculate_distance(o1["cent"], o2["cent"])
            now_state = "near" if dist < thresh_px else "far"
            pair_key = (min(id1, id2), max(id1, id2))
            prev_state = pair_state.get(pair_key, "far")

            # ---------- ① approach ----------
            if prev_state == "far" and now_state == "near":
                # ── (a) 記 approach 事件
                events_to_log.append({
                    "event_timestamp": current_ts,
                    "event_type": "approach",
                    "details": {
                        "id1": id1, "class1": o1["cls"],
                        "id2": id2, "class2": o2["cls"],
                        "distance_px": round(dist, 1)
                    },
                    "current_fps": fps_val,
                    "frame_w": f_w, "frame_h": f_h
                })
                # 假設 a 是人、b 是物（可自行判斷翻轉）, approach 時寫入 paired
                for it in recent_roi_enters:
                    if 'id' in it and it['id'] == id1:
                        it['paired'] = id2
                    elif 'id' in it and it['id'] == id2:
                        it['paired'] = id1
                        
                # ── (b) 若人-物進入 ROI 時間差 ≤ FRAME_WINDOW_SECOND → something_arrival
                enter_ts_a = enter_ts_b = None
                for it in recent_roi_enters:
                    if it.get("id") == id1:
                        enter_ts_a = it.get("enter_ts")
                    elif it.get("id") == id2:
                        enter_ts_b = it.get("enter_ts")
                
                mid_pt = (int((o1["cent"][0] + o2["cent"][0]) / 2),
                          int((o1["cent"][1] + o2["cent"][1]) / 2))
                # 都找到才比對
                if (enter_ts_a and enter_ts_b and
                    abs((enter_ts_a - enter_ts_b).total_seconds()) <= FRAME_WINDOW_SECOND and
                    pair_key not in something_arrival_pairs):

                    # 觸發事件 & 記錄不可重複
                    something_arrival_pairs.add(pair_key)
                    events_to_log.append({
                        "event_timestamp": current_ts,
                        "track_id": None,  # 到達事件不需要 track_id
                        "event_type": "arrival",
                        "details": {
                            "human_id": id1 if o1["cls"] in ('other_person','delivery_worker','food_delivery') else id2,
                            "thing_id": id2 if o1["cls"] in ('other_person','delivery_worker','food_delivery') else id1,
                            "delta_t_sec": round(abs((enter_ts_a - enter_ts_b).total_seconds()), 2)
                        },
                        "current_fps": fps_val,
                        "frame_w": f_w, "frame_h": f_h
                    })
                    # 畫面標註並加入 overlay
                    
                    cv2.putText(annotated_frame, "ARRIVAL",
                                (mid_pt[0] - 35, mid_pt[1] - 28),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                    arrival_overlay[pair_key] = current_ts + datetime.timedelta(seconds=ARRIVAL_DISPLAY_SEC)
                '''
                # ── (c) 原本的 approach 線段 & 文字
                cv2.line(annotated_frame, o1["cent"], o2["cent"], (0, 255, 255), 2)
                cv2.putText(annotated_frame, f"approach {dist:.1f}px",
                            (mid_pt[0] - 40, mid_pt[1] - 8),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                '''
                # 根據 now_state 給不同顏色
                if now_state == "near":
                    line_color = (0, 255, 255)  # 黃色
                else:
                    line_color = (0, 255, 0)    # 綠色
                cv2.line(annotated_frame, o1["cent"], o2["cent"], line_color, 2)
                cv2.putText(annotated_frame, f"dist: {dist:.1f}px",
                            (mid_pt[0] - 35, mid_pt[1] - 8),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, line_color, 2)

            # ---------- ② depart ----------
            elif prev_state == "near" and now_state == "far":
                events_to_log.append({
                    "event_timestamp": current_ts,
                    "event_type": "depart",
                    "details": {
                        "id1": id1, "class1": o1["cls"],
                        "id2": id2, "class2": o2["cls"],
                        "distance_px": round(dist, 1)
                    },
                    "current_fps": fps_val,
                    "frame_w": f_w, "frame_h": f_h
                })
                mid_pt = (int((o1["cent"][0] + o2["cent"][0]) / 2),
                          int((o1["cent"][1] + o2["cent"][1]) / 2))
                cv2.line(annotated_frame, o1["cent"], o2["cent"], (0, 255, 0), 2)
                cv2.putText(annotated_frame, f"depart {dist:.1f}px",
                            (mid_pt[0] - 35, mid_pt[1] - 8),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                # ---------- ③ 仍保持原狀態 ----------
                mid_pt = (int((o1["cent"][0] + o2["cent"][0]) / 2),
                          int((o1["cent"][1] + o2["cent"][1]) / 2))
                '''
                cv2.line(annotated_frame, o1["cent"], o2["cent"], (0, 255, 0), 2)
                cv2.putText(annotated_frame, f"dist: {dist:.1f}px",
                            (mid_pt[0] - 35, mid_pt[1] - 8),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                '''
                # 根據 now_state 給不同顏色
                if now_state == "near":
                    line_color = (0, 255, 255)  # 黃色
                else:
                    line_color = (0, 255, 0)    # 綠色
                cv2.line(annotated_frame, o1["cent"], o2["cent"], line_color, 2)
                cv2.putText(annotated_frame, f"dist: {dist:.1f}px",
                            (mid_pt[0] - 35, mid_pt[1] - 8),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, line_color, 2)

            # 更新狀態機
            pair_state[pair_key] = now_state

    # ── step-3：移除畫面上已不再出現的 pair 狀態 ─────────────────
    active_ids = set(ids)
    obsolete_pairs = [p for p in pair_state if p[0] not in active_ids or p[1] not in active_ids]
    for p in obsolete_pairs:
        pair_state.pop(p, None)

    return events_to_log


def draw_tracked_objects_and_stats(frame_to_draw_on, trk_objs, cls_names, current_roi, fps_val, current_frame_timestamp):
    # This function now MODIFIES frame_to_draw_on IN PLACE
    global total_frames_read_count, total_frames_processed_count, cumulative_detected_class_counts
    global object_loitering_start_time
    if current_roi: cv2.rectangle(frame_to_draw_on, (current_roi[0], current_roi[1]), (current_roi[2], current_roi[3]), (255,255,0),2); cv2.putText(frame_to_draw_on,"ROI",(current_roi[0],current_roi[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,0),2)
    for o in trk_objs:
        if len(o)==7:
            x1, y1, x2, y2, tid, cid, scr = map(float, o)
            x1, y1, x2, y2, tid, cid = int(x1), int(y1), int(x2), int(y2), int(tid), int(cid)
            cname = cls_names.get(cid, "Unk")
            clr = get_color_by_id(tid)
            cv2.rectangle(frame_to_draw_on, (x1, y1), (x2, y2), clr, 2)
            lbl = f"ID:{tid} {cname} {scr:.2f}"
            (lw, lh), bl = cv2.getTextSize(lbl, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            ly = max(lh + 5, y1 - 5)
            lx = x1
            cv2.rectangle(frame_to_draw_on, (lx, ly - lh - bl), (lx + lw, ly + bl), clr, cv2.FILLED)
            cv2.putText(frame_to_draw_on, lbl, (lx, ly), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            if object_loitering_start_time.get(tid) and isinstance(object_loitering_start_time[tid], datetime.datetime):
                # dur = (datetime.datetime.now() - object_loitering_start_time[tid]).total_seconds()
                dur = (current_frame_timestamp - object_loitering_start_time[tid]).total_seconds()
                cv2.putText(frame_to_draw_on, f"Loiter:{dur:.1f}s", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
    return frame_to_draw_on # Return the modified frame

def get_color_by_id(track_id): np.random.seed(track_id); return tuple(np.random.randint(0,255,size=3).tolist())

def save_event_log_final(log_data, filepath):
    try:
        existing_log = []
        if os.path.exists(filepath):
            with open(filepath, "r", encoding="utf-8") as f_in:
                try: existing_log = json.load(f_in)
                except json.JSONDecodeError: existing_log = []
                if not isinstance(existing_log, list): existing_log = []
        with open(filepath, "w", encoding="utf-8") as f_out:
            json.dump(existing_log + log_data, f_out, indent=4, ensure_ascii=False)
        print(f"Event log ({len(log_data)} new entries) appended to: {filepath}")
    except Exception as e: print(f"ERROR saving event log: {e}")



## Main Processing Function

In [24]:
import datetime
def main():
    global event_log, track_history, object_loitering_start_time, ROI, frame_buffer, active_clip_capture_tasks
    global total_frames_read_count, total_frames_processed_count, cumulative_detected_class_counts
    global object_in_roi_status, object_interaction_status, approach_depature_thresh_px, arrival_away_thresh_px
    global recent_roi_enters, recent_roi_exits
    
    event_log.clear(); track_history.clear(); object_loitering_start_time.clear(); ROI = None
    frame_buffer.clear(); active_clip_capture_tasks.clear()
    object_in_roi_status.clear(); object_interaction_status.clear()
    total_frames_read_count = 0; total_frames_processed_count = 0; cumulative_detected_class_counts.clear()
    recent_roi_enters.clear(); recent_roi_exits.clear()

    model_path = 'best.pt'
    local_video_path = INPUT_SOURCE # Make sure this video exists or provide a new one
    output_video_path = RUN_DIR / 'output_tracked_intent_boxmot_v8.mp4'
    conf_threshold = 0.3
    max_duration_sec = None # Set to None or a large number for full video processing
    
    # Create output directories if they don't exist
    if ENABLE_EVENT_CLIPS: Path(EVENT_CLIP_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    if ENABLE_EVENT_SNAPSHOTS: Path(EVENT_SNAPSHOT_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

    print(f"Loading model: {model_path}")
    if not Path(model_path).exists():
        print(f"Model {model_path} not found. Downloading yolov8n.pt.")
        try: torch.hub.download_url_to_file('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt', model_path); print("yolov8n.pt downloaded.")
        except Exception as e: print(f"Error downloading default model: {e}. Upload manually."); return
            
    try: model = YOLO(model_path); class_names_dict = model.names; print(f"Model loaded. Classes: {class_names_dict}")
    except Exception as e: print(f"ERROR loading YOLO model: {e}"); return

    if not Path(local_video_path).exists():
        print(f"ERROR: Video {local_video_path} not found. Please upload a video named 'sample_video.mp4' or change the path."); return

    cap = cv2.VideoCapture(local_video_path)
    if not cap.isOpened(): print(f"ERROR: Cannot open video: {local_video_path}"); return

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Video: {frame_width}x{frame_height} @ {fps:.2f} FPS")
    
    diag_len = (frame_width ** 2 + frame_height ** 2) ** 0.5
    approach_depature_thresh_px = diag_len * INTER_THRESH_RATIO
    arrival_away_thresh_px = diag_len * ARRIVAL_AWAY_DIST_RATIO

    frame_buffer = deque(maxlen=int(fps * FRAME_BUFFER_DURATION_SEC))
    print(f"Annotated frame buffer size: {frame_buffer.maxlen} frames ({FRAME_BUFFER_DURATION_SEC}s at {fps:.2f} FPS)")

    if ROI_MODE == 1:
        if frame_width > 2*ROI_MARGIN_PIXELS and frame_height > 2*ROI_MARGIN_PIXELS:
            ROI = (ROI_MARGIN_PIXELS, ROI_MARGIN_PIXELS, frame_width-ROI_MARGIN_PIXELS, frame_height-ROI_MARGIN_PIXELS)
        else: ROI = (0,0,frame_width,frame_height); print("WARN: Frame too small for margin, using full frame ROI.")
    elif ROI_MODE == 0: ROI = MANUAL_ROI
    else: ROI = (0,0,frame_width,frame_height); print("WARN: Invalid ROI_MODE, using full frame ROI.")
    if ROI: print(f"Using ROI: {ROI}")
    else: print("No ROI defined (MANUAL_ROI is None and ROI_MODE is not 1 or frame too small). Processing full frame for ROI checks.")

    tracker = None; current_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    try:
        tracker_type = 'bytetrack'
        tracker_config_path_cand = TRACKER_CONFIGS / (tracker_type + '.yaml') if isinstance(TRACKER_CONFIGS, Path) and (TRACKER_CONFIGS / (tracker_type + '.yaml')).exists() else None
        if not tracker_config_path_cand:
            import boxmot; pkg_cfg_path = Path(boxmot.__file__).parent/'configs'/(tracker_type+'.yaml')
            if pkg_cfg_path.exists(): tracker_config_path_cand = pkg_cfg_path
            else: raise FileNotFoundError(f"BoxMOT config for {tracker_type} not found.")
        print(f"Using BoxMOT tracker config: {tracker_config_path_cand}")
        tracker = create_tracker(tracker_type, tracker_config_path_cand, None, current_device, False, False)
        print(f"BoxMOT {tracker_type} tracker initialized on {current_device}.")
    except Exception as e: print(f"ERROR initializing BoxMOT: {e}. Using DummyTracker."); tracker = DummyTracker()
    if tracker is None: print("CRITICAL: Tracker is None. Aborting."); return

    out_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width,frame_height))
    
    processing_start_time = datetime.datetime.now()
    print("Starting video processing...")

    while cap.isOpened():
        ret, original_frame = cap.read()
        if not ret: print("End of video or read error."); break
        total_frames_read_count += 1
        # current_frame_timestamp = datetime.datetime.now()
        current_frame_timestamp = processing_start_time + datetime.timedelta(seconds=total_frames_read_count / fps)
        
        annotated_frame = original_frame.copy()

        yolo_results = model.predict(original_frame, conf=conf_threshold, verbose=False)
        detections_tensor = yolo_results[0].boxes.data
        if detections_tensor.numel() > 0:
            for cls_id in detections_tensor[:, 5].int().tolist(): cumulative_detected_class_counts[cls_id] += 1

        detections_np = np.empty((0,6))
        if isinstance(detections_tensor, torch.Tensor) and detections_tensor.numel() > 0:
            detections_np = detections_tensor.detach().cpu().numpy().astype("float32")

        tracked_dets_np = np.empty((0,7))
        if detections_np.shape[0] > 0 or not isinstance(tracker, DummyTracker):
            tracked_dets_np = tracker.update(detections_np, original_frame) 
            total_frames_processed_count +=1
        elif isinstance(tracker, DummyTracker):
            tracked_dets_np = tracker.update(None, original_frame); total_frames_processed_count +=1
        else: total_frames_processed_count +=1

        current_tracked_objects_list = []
        active_ids_this_frame = set()
        all_events_for_this_frame = []

        if tracked_dets_np.shape[0] > 0:
            for trk_data in tracked_dets_np:
                x1,y1,x2,y2,trk_id,conf,cls_id = trk_data[:7] # [x1, y1, x2, y2, track_id, score, cls_id]   # ← score 在第 6 欄
                # 先判斷回傳欄位數量 (7 or 8) 再正確拆包
                if trk_data.shape[0] == 8:       # 新版 BoxMOT
                    x1, y1, x2, y2, trk_id, conf, cls_id, _ = trk_data
                else:                            # 舊版 (7 欄) 或其他 tracker
                    x1, y1, x2, y2, trk_id, conf, cls_id     = trk_data
                # safety check：class id 不在字典就直接略過或指定 fallback
                cls_id_int = int(cls_id)
                if cls_id_int not in class_names_dict:
                    # 你可以 choose to:
                    continue                     # 1. 直接不處理這筆 (建議)
                    # or
                    # trk_class_name = "other"     # 2. 全部歸到 'other'
                else:
                    trk_class_name = class_names_dict[cls_id_int]

                current_tracked_objects_list.append([x1,y1,x2,y2,trk_id,cls_id,conf])
                active_ids_this_frame.add(int(trk_id))
                trk_centroid = get_centroid(trk_data); trk_class_name = class_names_dict.get(int(cls_id), "Unknown")
                if trk_class_name == "unknown": 
                    print(f"WARNING: Detected class ID {cls_id} not in class names dictionary. Using 'Unknown'.")
                   
                track_history[int(trk_id)].append((current_frame_timestamp, trk_centroid[0], trk_centroid[1], int(cls_id), conf))
                
                # Analyze behavior for this object (e.g., loitering, ROI entry/exit)
                # Pass annotated_frame here in case analyze_behavior needs to draw (though it currently doesn't)
                behavior_events = analyze_behavior(int(trk_id), track_history, [x1,y1,x2,y2], int(cls_id), trk_class_name, current_frame_timestamp, fps, ROI, frame_width, frame_height,annotated_frame)
                if behavior_events:
                    all_events_for_this_frame.extend(behavior_events)
        
        # Analyze interactions between all currently tracked objects for this frame
        # This function WILL draw on annotated_frame if interactions occur
        # interaction_events = analyze_interactions_for_frame(current_tracked_objects_list, current_frame_timestamp, annotated_frame, class_names_dict, fps, frame_width, frame_height)
        interaction_events = analyze_interactions_for_frame(
            trk_objs=current_tracked_objects_list,
            current_ts=current_frame_timestamp,
            annotated_frame=annotated_frame,
            cls_names=class_names_dict,    # id → 字串對照表
            fps_val=fps,
            f_w=frame_width,
            f_h=frame_height
        )
        if interaction_events:
            all_events_for_this_frame.extend(interaction_events)
        
        # Draw all general annotations (object boxes, stats, ROI) onto the annotated_frame
        # This happens AFTER interaction-specific annotations might have been drawn by analyze_interactions_for_frame
        draw_tracked_objects_and_stats(annotated_frame, current_tracked_objects_list, class_names_dict, ROI, fps, current_frame_timestamp)

        # Now, log all collected events for this frame, using the fully annotated_frame for media
        
        for event_data_item in all_events_for_this_frame:
            log_event(event_data_item, annotated_frame_for_media=annotated_frame)

        # Clean up history for tracks that are no longer active
        for inactive_id in list(track_history.keys() - active_ids_this_frame):
            if inactive_id in track_history: del track_history[inactive_id]
            if inactive_id in object_loitering_start_time: del object_loitering_start_time[inactive_id]
            if inactive_id in object_in_roi_status: del object_in_roi_status[inactive_id]
            
            # 清理互動狀態字典中與此ID相關的所有項目
            for key in list(object_interaction_status.keys()):
                if isinstance(key, tuple) and len(key) == 2 and inactive_id in key:
                    del object_interaction_status[key]
                elif isinstance(key, tuple) and len(key) == 4 and (key[1] == inactive_id or key[3] == inactive_id):
                    del object_interaction_status[key]
            '''
            for it in list(recent_roi_enters):
                if it['id'] == inactive_id:
                    recent_roi_enters.remove(it)
            '''
        
        # Add the fully ANNOTATED frame to the buffer for clip saving
        frame_buffer.append((annotated_frame.copy(), current_frame_timestamp))

        # Write the ANNOTATED frame to the output video
        out_writer.write(annotated_frame)

        # --- Handle active clip capture tasks (uses ANNOTATED frames from buffer) ---
        if ENABLE_EVENT_CLIPS:
            remaining_tasks = []
            for task in active_clip_capture_tasks:
                is_complete = False
                # Check if enough frames collected or if it's the end of the video
                if task['collected_frames']:
                    # Ensure frames are sorted by timestamp before checking end condition
                    task['collected_frames'].sort(key=lambda x: x[1])
                    last_collected_ts = task['collected_frames'][-1][1]
                    if last_collected_ts >= task['desired_clip_end_ts']:
                        is_complete = True
                
                # If processing has ended (not ret) and task has frames, consider it complete for saving
                if (not ret and task['collected_frames']) or is_complete:
                    frames_data_to_save = [f_data for f_data, ts_data in task['collected_frames'] 
                                           if ts_data >= task['desired_clip_start_ts'] and ts_data <= task['desired_clip_end_ts']]
                    if frames_data_to_save:
                         save_video_clip(frames_data_to_save, task['output_filename'], task['fps'], task['width'], task['height'])
                    # Mark as processed by not adding to remaining_tasks
                else:
                    # If not complete, keep collecting frames if current frame is within desired range
                    if current_frame_timestamp <= task['desired_clip_end_ts']:
                         # Only add if current frame is relevant to this task's time window
                         if current_frame_timestamp >= task['desired_clip_start_ts']:
                            # Check if frame already added (e.g. from pre-fill)
                            if not any(f_ts == current_frame_timestamp for _, f_ts in task['collected_frames']):
                                task['collected_frames'].append((annotated_frame.copy(), current_frame_timestamp))
                    remaining_tasks.append(task)
            active_clip_capture_tasks = remaining_tasks

        
        
        if max_duration_sec and (time.time() - processing_start_time > max_duration_sec): print(f"Max duration {max_duration_sec}s reached."); break
        if total_frames_read_count % 100 == 0: print(f"Processed {total_frames_read_count} frames...")
    
    # Cleanup and finalize
    cap.release(); out_writer.release()
    
    
    # Save event log
    if event_log:
        save_event_log_final(event_log, EVENT_LOG_FILE)
        print(f"\n--- Event Log Summary ---")
        event_types = {}
        for e in event_log: event_types[e['event_type']] = event_types.get(e['event_type'], 0) + 1
        for et, count in sorted(event_types.items()): print(f"{et}: {count} events")
    else: print("No events were logged.")
    
    # --- Event Clips Status (原樣保留) --------------------------------------
    print(f"\n--- Event Clips ({EVENT_CLIP_OUTPUT_DIR}/) Status ---")
    if Path(EVENT_CLIP_OUTPUT_DIR).is_dir():
        clips = list(Path(EVENT_CLIP_OUTPUT_DIR).glob("*.mp4"))
        print(f"Found {len(clips)} clips in {EVENT_CLIP_OUTPUT_DIR}.")
    else:
        print(f"Event clips directory {EVENT_CLIP_OUTPUT_DIR} not found.")
    
    # --- Event Snapshots Status (原樣保留) --------------------------------------
    print(f"\n--- Event Snapshots ({EVENT_SNAPSHOT_OUTPUT_DIR}/) Status ---")
    if Path(EVENT_SNAPSHOT_OUTPUT_DIR).is_dir():
        snaps = list(Path(EVENT_SNAPSHOT_OUTPUT_DIR).glob("*.jpg"))
        print(f"Found {len(snaps)} snapshots in {EVENT_SNAPSHOT_OUTPUT_DIR}.")
    else:
        print(f"Event snapshots directory {EVENT_SNAPSHOT_OUTPUT_DIR} not found.")

    print(f"\n--- Processing Complete ---")
    print(f"Total frames read: {total_frames_read_count}")
    print(f"Total frames processed: {total_frames_processed_count}")
    print(f"Output video saved to: {output_video_path}")

In [25]:
main()


Loading model: best.pt
Model loaded. Classes: {0: 'package', 1: 'bag', 2: 'other_person', 3: 'delivery_worker', 4: 'food_delivery'}
Video: 1280x720 @ 30.00 FPS
Annotated frame buffer size: 300 frames (10s at 30.00 FPS)
Using ROI: (0, 250, 650, 720)
Using BoxMOT tracker config: C:\Users\user\venv_tracker\Lib\site-packages\boxmot\configs\bytetrack.yaml
BoxMOT bytetrack tracker initialized on cpu.
Starting video processing...
Processed 100 frames...
Event snapshot saved: output\golden_sample\20250601_134922\snapshots\roi_enter_id100_20250601_135005_270.jpg
Processed 200 frames...
Event snapshot saved: output\golden_sample\20250601_134922\snapshots\roi_enter_id102_20250601_135008_537.jpg
Event snapshot saved: output\golden_sample\20250601_134922\snapshots\approach_20250601_135008_837.jpg
[LINE-Bot] 圖片已推送到 LINE: https://storage.googleapis.com/cv_event_image/event_images/arrival_20250601_135208.jpg?X-Goog-Algorithm=GOOG4-RSA-SHA256&X-Goog-Credential=644134838534-compute%40developer.gservicea

In [26]:
from pathlib import Path
import json
from collections import Counter, defaultdict
import pandas as pd      # ✅ 用來排版統計表

# --- Review Event Logs, Clips, and Snapshots (Enhanced) --------------------
print(f"\n--- Event Log ({EVENT_LOG_FILE}) Status ---")
object_counter   = Counter()                 # 物件類別 → 出現次數
event_counter    = Counter()                 # 事件類別 → 出現次數
obj_event_matrix = defaultdict(Counter)      # 物件類別 → (事件類別 → 次數)

if Path(EVENT_LOG_FILE).exists():
    try:
        with open(EVENT_LOG_FILE, "r", encoding="utf-8") as f:
            logged_events_content = json.load(f)

        total_events = len(logged_events_content)
        print(f"Found {total_events} events in {EVENT_LOG_FILE}.")

        # ▍統計迴圈
        for evt in logged_events_content:
            # 1. 事件類別 (可依實際欄位名稱增減備援鍵)
            evt_type = evt.get("event_type") or evt.get("event") or evt.get("type") or "<unknown>"
            event_counter[evt_type] += 1

            # 2. 物件類別 (單一字串或 list 皆可)
            raw_obj = evt.get("class_name") or evt.get("class") or evt.get("object_classes")
            obj_classes = raw_obj if isinstance(raw_obj, list) else [raw_obj or "<event>"]

            for cls in obj_classes:
                object_counter[cls] += 1
                obj_event_matrix[cls][evt_type] += 1

        # ▍輸出統計表 -------------------------------------------------------
        obj_df  = (pd.DataFrame(object_counter.items(), columns=["Object Class", "Count"])
                     .sort_values("Count", ascending=False))
        evt_df  = (pd.DataFrame(event_counter.items(),  columns=["Event Type",  "Count"])
                     .sort_values("Count", ascending=False))
        cross_df = (pd.DataFrame(obj_event_matrix).fillna(0).astype(int).T
                      .loc[obj_df["Object Class"]])   # 依物件出現頻次排序

        print("\n=== Object Class Distribution ===")
        print(obj_df.to_string(index=False))

        print("\n=== Event Type Distribution ===")
        print(evt_df.to_string(index=False))

        print("\n=== Object × Event Crosstab ===")
        print(cross_df.to_string())
    except Exception as e:
        print(f"Error reading event log: {e}")
else:
    print(f"Event log file {EVENT_LOG_FILE} not found.")

# --- Event Clips Status (原樣保留) -----------------------------------------
print(f"\n--- Event Clips ({EVENT_CLIP_OUTPUT_DIR}/) Status ---")
if Path(EVENT_CLIP_OUTPUT_DIR).is_dir():
    clips = list(Path(EVENT_CLIP_OUTPUT_DIR).glob("*.mp4"))
    print(f"Found {len(clips)} video clips in {EVENT_CLIP_OUTPUT_DIR}.")
else:
    print(f"Event clips directory {EVENT_CLIP_OUTPUT_DIR} not found.")

# --- Event Snapshots Status (原樣保留) --------------------------------------
print(f"\n--- Event Snapshots ({EVENT_SNAPSHOT_OUTPUT_DIR}/) Status ---")
if Path(EVENT_SNAPSHOT_OUTPUT_DIR).is_dir():
    snaps = list(Path(EVENT_SNAPSHOT_OUTPUT_DIR).glob("*.jpg"))
    print(f"Found {len(snaps)} snapshots in {EVENT_SNAPSHOT_OUTPUT_DIR}.")
else:
    print(f"Event snapshots directory {EVENT_SNAPSHOT_OUTPUT_DIR} not found.")




--- Event Log (output\golden_sample\20250601_134922\golden_sample_events.json) Status ---
Found 74 events in output\golden_sample\20250601_134922\golden_sample_events.json.

=== Object Class Distribution ===
 Object Class  Count
      <event>     29
 other_person     25
food_delivery     17
      package      3

=== Event Type Distribution ===
Event Type  Count
 roi_enter     29
  approach     16
    depart     12
 loitering      9
  roi_exit      4
      away      3
   arrival      1

=== Object × Event Crosstab ===
               roi_enter  loitering  roi_exit  away  approach  arrival  depart
<event>                0          0         0     0        16        1      12
other_person          13          5         4     3         0        0       0
food_delivery         14          3         0     0         0        0       0
package                2          1         0     0         0        0       0

--- Event Clips (output\golden_sample\20250601_134922\clips/) Status ---
Found 0